In [1]:
import pandas as pd
from operator import attrgetter
from datetime import timedelta
from datetime import datetime
from calendar import monthrange

In [2]:
customers_dataset = pd.read_csv("olist_customers_dataset.csv")
#таблица с уникальными идентификаторами пользователей

In [3]:
orders_dataset = pd.read_csv("olist_orders_dataset.csv", parse_dates=[3, 4, 5, 6, 7])
#таблица заказов

<!-- Уникальные статусы заказов в таблице olist_orders_dataset:

created —  создан
approved —  подтверждён
invoiced —  выставлен счёт
processing —  в процессе сборки заказа
shipped —  отгружен со склада
delivered —  доставлен пользователю
unavailable —  недоступен
canceled —  отменён -->

# Уникальные статусы заказов в таблице olist_orders_dataset:

# created —  создан
# approved —  подтверждён
# invoiced —  выставлен счёт
# processing —  в процессе сборки заказа
# shipped —  отгружен со склада
# delivered —  доставлен пользователю
# unavailable —  недоступен
# canceled —  отменён

In [4]:
orders_dataset.groupby("order_status", as_index = False).agg({"customer_id": "count"})
#посмотрим  на распределение заказов по статусам

,order_status,customer_id
0,approved,2
1,canceled,625
2,created,5
3,delivered,96478
4,invoiced,314
5,processing,301
6,shipped,1107
7,unavailable,609


In [5]:
order_items_dataset = pd.read_csv("olist_order_items_dataset.csv")
#товарные позиции, входящие в заказы

In [6]:
order_items_dataset.product_id.nunique()
#количество уникальных наименований товаров

32951

In [7]:
order_items_dataset.groupby("order_id", as_index = False).agg({"order_item_id": "count"}) \
        .sort_values("order_item_id", ascending=False).head(10)
#смотрим, сколько позиций товара в каждом чеке

,order_id,order_item_id
50137,8272b63d03f5f79c56e9e4120aec44ef,21
10459,1b15974a0141d54e36626dca3fdc731a,20
65715,ab14fdcfbe524636d65ee38360e22ce8,20
60941,9ef13efd6949e4573a18964dd1bbe7f5,15
25583,428a2f660dc84138d969ccd69a0ab6d5,15
59688,9bdc4d4c71aa1de4606060929dee888c,14
44594,73c8ab38f07dc94389065f7eba4f297a,14
21484,37ee401157a3a0b28c9c6d0ed8c3b24b,13
16854,2c2a19b5703863c908512d135aa6accc,12
74094,c05d6a79e55da72ca780ce90364abed9,12


In [8]:
customers_orders_dataset = customers_dataset.merge(orders_dataset)
# обьединим 2 таблицы: с заказами и с покупателями

# 1. Сколько у нас пользователей, которые совершили покупку только один раз? (7 баллов)

   Для успешного решения задания необходимо понять, заказы с какими статусами мы будем считать покупкой. Заказы со статусами "unavailable" и "canceled" невозможно считать покупками, так как они отменены или недоступны.
Заказы со статусом "created" мы не можем считать покупкой, так как на этом этапе потенциальный покупатель только создал заказ.
Та же самая история со статусом "invoiced" — выставлен счёт - если потенцииальный покупатель не оплатит заказ, то покупка не состоится.

   Статус "processing" — в процессе сборки заказа - потенцииальный покупатель нажал заказать, после чего выставлен счет, счет оплачен и после этого продавец собирает заказ, считаем, что покупка совершена.
   Следующий статус "shipped" - продавец отправил заказ. 
   "delivered" — доставлен пользователю - заказы с таким статусом точно подходят под наше условие.

   Итого, будем считать, что все заказы со статусами: processing, shipped, delivered - оплачены и не отменены, те есть покупатель совершил покупку.

In [9]:
# В первой строчке отбираем заказы, которые, как мы выяснили выше, будем считать за совершенную покупку 
# во второй - группируем по полю "customer_unique_id"
# в третьей - делаем агрегацию подсчета по полю "customer_zip_code_prefix"
# в четвертой - отбираем строчки в которых по полю "customer_zip_code_prefix" стоит один заказ, смотрим объем итоговой таблицы
customers_orders_dataset.query("order_status == ['processing', 'shipped', 'delivered']") \
                .groupby("customer_unique_id", as_index=False) \
                .agg({"customer_zip_code_prefix": "count"}) \
                .query("customer_zip_code_prefix  == 1").shape

(91814, 2)

In [10]:
# Итого 91814 пользователей, которые совершили покупку только один раз

# 2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)? (5 баллов)

In [11]:
# Используя предыдущий опыт анализа, в качестве недоставленного, но совершенного заказа мы будем считать заказ, который
# имеет статусы "processing", "shipped", "delivered", "unavailable", "canceled", при этом в поле "order_delivered_customer_date"
# отсутствует какая либо-дата доставки

In [12]:
orders_dataset.query("order_status == ['processing', 'shipped', 'delivered', 'unavailable', 'canceled']").order_delivered_customer_date.isna().sum()

2644

In [13]:
#создадим новый датафрейм с недоставленными заказами
no_delivered = orders_dataset[orders_dataset.order_delivered_customer_date.isna()] \
                .query("order_status == ['processing', 'shipped', 'delivered', 'unavailable', 'canceled']")

In [14]:
# приведём даты к месяцам, т.к. нас интересует дальнейшая грппировка по месяцам
no_delivered.order_purchase_timestamp = no_delivered.order_purchase_timestamp.dt.strftime('%Y-%m')

In [15]:
#посчитаем кол-во недоставленных заказов по каждой причине за каждый месяц, далее посчитаем среднее значение для каждой причины
no_delivered.groupby(["order_purchase_timestamp", "order_status"], as_index=False) \
            .agg({"customer_id": "count"}) \
            .groupby("order_status") \
            .agg({"customer_id": "mean"})

,customer_id
order_status,
canceled,25.791667
delivered,2.000000
processing,15.842105
shipped,48.130435
unavailable,29.000000


# 3. По каждому товару определить, в какой день недели товар чаще всего покупается. (5 баллов)

In [16]:
#склеим два датафрейма с покупками и датами
df = orders_dataset.merge(order_items_dataset)

In [17]:
df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,1,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,1,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,1,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,1,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72


In [18]:
#Основываясь на ранее проведенном анализе, оставим только те строки, в которых статус заказа мы считаем покупкой
purchase_df = df.query("order_status == ['processing', 'shipped', 'delivered']")

In [19]:
purchase_df.dtypes

order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
order_item_id                             int64
product_id                               object
seller_id                                object
shipping_limit_date                      object
price                                   float64
freight_value                           float64
dtype: object

In [46]:
# Мы видим 5 полей с датой в нашем датафрейме "purchase_df". Нам надо найти самое ранне время заказа, по которому мы можем
# сказать, что заказ совершен. Датой покупки товара будем считать поле "order_approved_at" — 
# время подтверждения оплаты заказа, так как имеенно по это дате мы можем понять, что покупка совершена.
# Поле "order_purchase_timestamp" —  время создания заказа, как мы ранее выяснили не может свидетельствовать о совершенной
# покупке.
purchase_df.order_approved_at = purchase_df.order_approved_at.dt.day_name()

AttributeError: Can only use .dt accessor with datetimelike values

In [21]:
purchase_df.groupby(["product_id", "order_approved_at"], as_index=False) \
    .agg({"customer_id": "count"}) \
    .sort_values("customer_id", ascending=False)

,product_id,order_approved_at,customer_id
40699,aca2eb7d00ea1a7b8ebd4e68314663af,Tuesday,119
19963,53b36df67ebb7c41585e8d54d6772e08,Tuesday,105
40694,aca2eb7d00ea1a7b8ebd4e68314663af,Friday,96
40700,aca2eb7d00ea1a7b8ebd4e68314663af,Wednesday,89
15791,422879e10f46682990de24d770e7f83d,Tuesday,89
...,...,...,...
23773,64051126507bdb875c1baf5b3b5a9a98,Tuesday,1
23774,64051126507bdb875c1baf5b3b5a9a98,Wednesday,1
23776,6405e17a3e8718d5316c5d644320f253,Saturday,1
23777,6405e17a3e8718d5316c5d644320f253,Sunday,1


# 4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? Не стоит забывать, что внутри месяца может быть не целое количество недель. Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть. (8 баллов)

In [22]:
# Соединим все три изначальных датафрейма в один
full_DF = customers_orders_dataset.merge(order_items_dataset)

In [23]:
full_DF.order_approved_at = full_DF.order_approved_at.dt.month_name()
# переводим дату в названия месяца

In [24]:
full_DF

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,May,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05,1,a9516a079e37a9c9c36b9b78b10169e8,7c67e1448b00f6e969d365cea6b010ab,2017-05-22 15:22:12,124.99,21.88
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,January,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-01-18 20:58:32,289.00,46.48
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,May,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13,1,bd07b66896d6f1494f5b86251848ced7,7c67e1448b00f6e969d365cea6b010ab,2018-06-05 16:19:10,139.94,17.79
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,March,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10,1,a5647c44af977b148e0a3a4751a09e2e,7c67e1448b00f6e969d365cea6b010ab,2018-03-27 16:31:16,149.94,23.36
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,July,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15,1,9391a573abe00141c56e38d84d7d5b3b,4a3ca9315b744ce9f8e9374361493884,2018-07-31 10:10:09,230.00,22.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112645,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP,6760e20addcf0121e9d58f2f1ff14298,delivered,2018-04-07 15:48:17,April,2018-04-11 02:08:36,2018-04-13 20:06:37,2018-04-25,1,ccb4503d9d43d245d3b295d0544f988b,527801b552d0077ffd170872eb49683b,2018-04-12 16:08:45,74.90,13.88
112646,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,2018-04-04 08:20:22,April,2018-04-05 18:42:35,2018-04-11 18:54:45,2018-04-20,1,9ede6b0570a75a4b9de4f383329f99ee,3fd1e727ba94cfe122d165e176ce7967,2018-04-10 08:35:12,114.90,14.16
112647,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE,fed4434add09a6f332ea398efd656a5c,delivered,2018-04-08 20:11:50,April,2018-04-09 17:52:17,2018-05-09 19:03:15,2018-05-02,1,7a5d2e1e131a860ae7d18f6fffa9d689,d9e7e7778b32987280a6f2cb9a39c57d,2018-04-12 20:30:03,37.00,19.04
112648,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS,e31ec91cea1ecf97797787471f98a8c2,delivered,2017-11-03 21:08:33,November,2017-11-06 18:24:41,2017-11-16 19:58:39,2017-12-05,1,f819f0c84a64f02d3a5606ca95edd272,4869f7a5dfa277a7dca6462dcf3b52b2,2017-11-09 21:15:51,689.00,22.07


In [25]:
# вычисяем количество недель в каждом месяце, в котором совершались покупки
full_DF['weeks'] = orders_dataset.order_approved_at.dt.days_in_month / 7

In [26]:
customer_by_month = full_DF.query("order_status == ['processing', 'shipped', 'delivered']") \
    .groupby(['customer_unique_id','order_approved_at','weeks'], as_index=False) \
    .agg({'order_status':'count'}) \
    .groupby(['customer_unique_id','order_approved_at','weeks'],as_index=False) \
    .agg({'order_status':'sum'}) \
    .rename(columns={'order_approved_at':'month','order_status':'avg_orders'}) 
# ставим фильтр по статусам заказов, которые мы признаем покупками
# группируем данные для расчета среднего кол-ва выполненых заказов  

In [27]:
customer_by_month.avg_orders = customer_by_month.avg_orders / customer_by_month.weeks
# считаем среднее количество покупок в неделю (по месяцам)

In [28]:
customer_by_month

,customer_unique_id,month,weeks,avg_orders
0,0000366f3b9a7992bf8c76cfdf3221e2,May,4.428571,0.225806
1,0000b849f77a49e4a4ce2b2a4ca5be3f,May,4.285714,0.233333
2,0000f46a3911fa3c0805444483337064,March,4.000000,0.250000
3,0000f6ccb0745a6a4b88665a16c9f078,October,4.428571,0.225806
4,0004aac84e0df4da2b147fca70cf8255,November,4.428571,0.225806
...,...,...,...,...
90953,fffbf87b7a1a6fa8b03f081c5f51a201,December,4.428571,0.225806
90954,fffcc512b7dfecaffd80f13614af1d16,April,4.285714,0.233333
90955,fffea47cd6d3cc0a88bd621562a9d061,December,4.428571,0.225806
90956,ffff371b4d645b6ecea244b27531430a,February,4.428571,0.225806


# 5. Используя pandas, проведи когортный анализ пользователей. В период с января по декабрь выяви когорту с самым высоким retention на 3й месяц. Описание подхода можно найти тут. (10 баллов)

In [29]:
# Retention дня X — это процент пользователей, которые вернутся к продукту в день X.
# Иными словами, если в какой-то конкретный день (день 0) пришло 100 новых пользователей, а на 10-ый день вернулось 10,
# то Retention 10-го дня составит 10 / 100 = 10%.

In [30]:
retention_df = customers_orders_dataset.query("order_status == ['processing', 'shipped', 'delivered']")
# создадим датафрейм для задания и поставим фильтр по статусам заказов, которые мы признаем покупками

In [31]:
# Добавим колонку с датой первой покупки
retention_df["first_buy"] = retention_df.groupby('customer_unique_id')['order_approved_at'].transform('min')

C:\Users\adeptt\AppData\Local\Temp\ipykernel_7044\3437843809.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retention_df["first_buy"] = retention_df.groupby('customer_unique_id')['order_approved_at'].transform('min')


In [32]:
retention_df.first_buy.min()
# посмотрим самую раннюю дату первой покупки

Timestamp('2016-09-15 12:16:38')

In [33]:
retention_df.first_buy.max()
# посмотрим самую позднюю дату первой покупки

Timestamp('2018-08-29 15:10:26')

In [34]:
# Мы видим, что полный год по которому мы можем сделать анализ - это 2017 г., так как по другим годам нет полных данных.

In [35]:
# Колонки с датами, по которым мы будем работать, необходимо привести к месяцам, т.к. нас интересуют данные в разрезе месяца
retention_df.order_approved_at = retention_df.order_approved_at.dt.strftime('%Y-%m')
retention_df.first_buy = retention_df['first_buy'].dt.strftime('%Y-%m')

C:\Users\adeptt\AppData\Local\Temp\ipykernel_7044\2607369243.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retention_df.order_approved_at = retention_df.order_approved_at.dt.strftime('%Y-%m')
C:\Users\adeptt\AppData\Local\Temp\ipykernel_7044\2607369243.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retention_df.first_buy = retention_df['first_buy'].dt.strftime('%Y-%m')


In [36]:
# отбираем необходимые даты первой покупки, группируем датафрейм по датам и считаем количество уникальных айди
retention_df = retention_df.query("'2017-01' <= first_buy <= '2017-12'") \
    .groupby(["order_approved_at", "first_buy"], as_index=False) \
    .agg({"customer_unique_id": "nunique"})

In [37]:
retention_df

,order_approved_at,first_buy,customer_unique_id
0,2017-01,2017-01,705
1,2017-02,2017-01,4
2,2017-02,2017-02,1661
3,2017-03,2017-01,2
4,2017-03,2017-02,4
...,...,...,...
166,2018-08,2017-08,6
167,2018-08,2017-09,3
168,2018-08,2017-10,9
169,2018-08,2017-11,5


In [38]:
# Переведем колонки с датами в номер месяца
retention_df["first_buy"] = pd.to_datetime(retention_df["first_buy"]).dt.to_period("m")
retention_df["order_approved_at"] = pd.to_datetime(retention_df["order_approved_at"]).dt.to_period("m")

In [39]:
# вычтем одну дату из другой для получения количества месяцев между двумя датами
retention_df['period'] = (retention_df.order_approved_at - retention_df.first_buy).apply(attrgetter("n"))

In [40]:
retention_df.query("period == 3")
# оставим только те строки, в которых в колонке period значение 3

,order_approved_at,first_buy,customer_unique_id,period
6,2017-04,2017-01,1,3
11,2017-05,2017-02,3,3
17,2017-06,2017-03,9,3
24,2017-07,2017-04,4,3
32,2017-08,2017-05,14,3
41,2017-09,2017-06,12,3
50,2017-10,2017-07,7,3
61,2017-11,2017-08,11,3
73,2017-12,2017-09,14,3
86,2018-01,2017-10,4,3


In [41]:
# значение 17 нам не подходит, так как вторая дата интервала за пределами 2017 г., поэтому мы находим 2 подходящих когорты
# с месяцами первой покупки май и сентябрь 2017 г.

# 6. Часто для качественного анализа аудитории используют подходы, основанные на сегментации. Используя python, построй RFM-сегментацию пользователей, чтобы качественно оценить свою аудиторию. В кластеризации можешь выбрать следующие метрики: R - время от последней покупки пользователя до текущей даты, F - суммарное количество покупок у пользователя за всё время, M - сумма покупок за всё время. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, frequency от 2 до 5 заказов в неделю, monetary от 1780 до 3560 рублей в неделю. Описание подхода можно найти тут. (35 баллов)

In [89]:
# создадим датасет для нашей задачи
df_for_RFM = customers_orders_dataset.merge(order_items_dataset)

In [90]:
df_for_RFM

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05,1,a9516a079e37a9c9c36b9b78b10169e8,7c67e1448b00f6e969d365cea6b010ab,2017-05-22 15:22:12,124.99,21.88
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-01-18 20:58:32,289.00,46.48
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13,1,bd07b66896d6f1494f5b86251848ced7,7c67e1448b00f6e969d365cea6b010ab,2018-06-05 16:19:10,139.94,17.79
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10,1,a5647c44af977b148e0a3a4751a09e2e,7c67e1448b00f6e969d365cea6b010ab,2018-03-27 16:31:16,149.94,23.36
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15,1,9391a573abe00141c56e38d84d7d5b3b,4a3ca9315b744ce9f8e9374361493884,2018-07-31 10:10:09,230.00,22.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112645,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP,6760e20addcf0121e9d58f2f1ff14298,delivered,2018-04-07 15:48:17,2018-04-07 16:08:45,2018-04-11 02:08:36,2018-04-13 20:06:37,2018-04-25,1,ccb4503d9d43d245d3b295d0544f988b,527801b552d0077ffd170872eb49683b,2018-04-12 16:08:45,74.90,13.88
112646,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,2018-04-04 08:20:22,2018-04-04 08:35:12,2018-04-05 18:42:35,2018-04-11 18:54:45,2018-04-20,1,9ede6b0570a75a4b9de4f383329f99ee,3fd1e727ba94cfe122d165e176ce7967,2018-04-10 08:35:12,114.90,14.16
112647,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE,fed4434add09a6f332ea398efd656a5c,delivered,2018-04-08 20:11:50,2018-04-08 20:30:03,2018-04-09 17:52:17,2018-05-09 19:03:15,2018-05-02,1,7a5d2e1e131a860ae7d18f6fffa9d689,d9e7e7778b32987280a6f2cb9a39c57d,2018-04-12 20:30:03,37.00,19.04
112648,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS,e31ec91cea1ecf97797787471f98a8c2,delivered,2017-11-03 21:08:33,2017-11-03 21:31:20,2017-11-06 18:24:41,2017-11-16 19:58:39,2017-12-05,1,f819f0c84a64f02d3a5606ca95edd272,4869f7a5dfa277a7dca6462dcf3b52b2,2017-11-09 21:15:51,689.00,22.07


In [91]:
df_for_RFM = df_for_RFM.query("order_status == ['processing', 'shipped', 'delivered']").dropna()
#Удалим колонки с пропущенными значениями и оставим только те заказы, которые мы считаем покупкой (с соответсвующими статусами
# заказа)

In [96]:
# возьмем нужные для RFM анализа колонки
df_for_RFM = df_for_RFM.groupby(['customer_unique_id', 'order_id', 'order_purchase_timestamp'], as_index = False) \
               .agg({'price': 'sum'})

In [98]:
# Переведем датувремя в дату, работать будем по колонке "order_purchase_timestamp", так как в датафрейме имеются заказы,
# со статусами, которые свидетельствуют о покупке.
df_for_RFM['order_purchase_timestamp'] = pd.to_datetime(df_for_RFM['order_purchase_timestamp']).dt.date

In [100]:
# Поскольку датафрейм достаточно старый (2016-2018 гг.) нам нужно определить, что мы будем понимать под "текущей датой"
# в смысле задания. Текущей датой мы будем считать последнюю по времени дату создания заказа
Current_date = df_for_RFM.order_purchase_timestamp.max()

In [101]:
# Создаем RFM таблицу. Для этого группируем по уникальному ID клиента, и агририруем (из текущей даты вычитаем дату последней 
# покупки клиента, считаем количество заказов на одного клиента и берем сумму, которую потратил каждый клиент).
RFM=df_for_RFM.groupby('customer_unique_id', as_index = False) \
                .agg({'order_purchase_timestamp': lambda x: (Current_date - x.max()).days, # Recency
                                                                      'order_id': lambda x: len(x.unique()), # Frequency
                                                                      'price': lambda x: x.sum()}) # Monetary

In [104]:
# Назовем колонки согласно номенклатуре RFM
RFM.rename(columns={'order_purchase_timestamp': 'recency', 
                                      'order_id': 'frequency',
                                      'price': 'monetary'}, inplace=True)

In [105]:
RFM

,customer_unique_id,recency,frequency,monetary
0,0000366f3b9a7992bf8c76cfdf3221e2,111,1,129.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,114,1,18.90
2,0000f46a3911fa3c0805444483337064,537,1,69.00
3,0000f6ccb0745a6a4b88665a16c9f078,321,1,25.99
4,0004aac84e0df4da2b147fca70cf8255,288,1,180.00
...,...,...,...,...
93331,fffcf5a5ff07b0908bd4e2dbc735a684,447,1,1570.00
93332,fffea47cd6d3cc0a88bd621562a9d061,262,1,64.89
93333,ffff371b4d645b6ecea244b27531430a,568,1,89.90
93334,ffff5962728ec6157033ef9805bacc48,119,1,115.00


In [109]:
RFM.describe()

,recency,frequency,monetary
count,93336.000000,93336.000000,93336.000000
mean,237.440945,1.033417,141.621663
std,152.556574,0.209098,215.714143
min,0.000000,1.000000,0.850000
25%,114.000000,1.000000,47.650000
50%,218.000000,1.000000,89.700000
75%,346.000000,1.000000,154.692500
max,713.000000,15.000000,13440.000000


In [106]:
# Для понимания на какие сегменты делить пользователей по количеству покупок, сделаем разведку 
RFM.groupby('frequency') \
         .agg({'customer_unique_id': 'count'})

,customer_unique_id
frequency,
1,90536
2,2572
3,181
4,28
5,9
6,5
7,3
9,1
15,1


In [110]:
# Будем делить пользователей на группы по процентилю с учетом того, что 1 - наихудший показатель, 3 - наилучший показатель:
# recency разобьем на группы по процентилям 0-25, 25-75 и 75: 3 - от 0 до 114 дней; 2 - от 115 до 346 дней; и 1 - больше 352
# frequency целесообразно разбить на группы по другому: 1. 1 покупка; 2. 2-4 покупки; 3. 5 и больше покупок
# monetary - поделим по процентилям 0-25, 25-75 и 75 и далее: 1. 0 - 48 денег; 2. 49 денег - 155 денег; 3. 156 денег и более

In [116]:
# Для значений recency и monetary
quantiles1 = RFM.quantile(q=[0.25, 0.75])

In [117]:
# Для значений frequency
quantiles2 = RFM.quantile(q=[0.98, 0.9997])

In [118]:
# Строим 3 разные функции с условиями.

# Создаем функцию для Recency
def R_Class(x,p,d):
    if x <= d[p][0.25]:
        return 3
    elif x <= d[p][0.75]:
        return 2
    else:
        return 1
    
# Создаем функцию для Frequency
def F_Class(x,p,d):
    if x < d[p][0.98]:
        return 1
    elif x <= d[p][0.9997]:
        return 2
    else:
        return 3

# Создаем функцию для Monetary    
def M_Class(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.75]:
        return 2
    else:
        return 3

RFM['R_seg'] = RFM['recency'].apply(R_Class, args=('recency', quantiles1,))
RFM['F_seg'] = RFM['frequency'].apply(F_Class, args=('frequency', quantiles2,))
RFM['M_seg'] = RFM['monetary'].apply(M_Class, args=('monetary', quantiles1,))
RFM['RFMClass'] = RFM.R_seg.map(str) + RFM.F_seg.map(str) + RFM.M_seg.map(str)

In [119]:
RFM

,customer_unique_id,recency,frequency,monetary,R_seg,F_seg,M_seg,RFMClass
0,0000366f3b9a7992bf8c76cfdf3221e2,111,1,129.90,3,1,2,312
1,0000b849f77a49e4a4ce2b2a4ca5be3f,114,1,18.90,3,1,1,311
2,0000f46a3911fa3c0805444483337064,537,1,69.00,1,1,2,112
3,0000f6ccb0745a6a4b88665a16c9f078,321,1,25.99,2,1,1,211
4,0004aac84e0df4da2b147fca70cf8255,288,1,180.00,2,1,3,213
...,...,...,...,...,...,...,...,...
93331,fffcf5a5ff07b0908bd4e2dbc735a684,447,1,1570.00,1,1,3,113
93332,fffea47cd6d3cc0a88bd621562a9d061,262,1,64.89,2,1,2,212
93333,ffff371b4d645b6ecea244b27531430a,568,1,89.90,1,1,2,112
93334,ffff5962728ec6157033ef9805bacc48,119,1,115.00,2,1,2,212


In [121]:
# отберем 5 значений RFMClass по количеству пользователей
RFM.value_counts('RFMClass').head(5)

RFMClass
212    22907
312    11488
211    11433
112    11249
213    10764
dtype: int64

In [122]:
# Чаще всего втречается следующее поведение покупателей: 1 покупка, срок покупки от 3 мес. до года, при этом наиболее
# часто встречается средний чек покупки. 